<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/text_analysis_coref60k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download ru_core_news_sm
!python -m spacy download ru_core_news_md

     |████████████████████████████████| 1.6MB 6.9MB/s 
     |████████████████████████████████| 788kB 19.2MB/s 
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Found existing installation: setuptools 56.1.0
    Uninstalling setuptools-56.1.0:
      Successfully uninstalled setuptools-56.1.0


     |████████████████████████████████| 12.8 MB 82 kB/s 
     |████████████████████████████████| 9.1 MB 35.5 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 456 kB 35.7 MB/s 
     |████████████████████████████████| 1.1 MB 32.4 MB/s 
     |████████████████████████████████| 113 kB 58.0 MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=cfc791a02e40e3a4d227f362cf0bb854afcf47335cd827a915efd35b3442f9cd
  Stored in directory: /root/.cache/pip/wheels/83/a6/12/bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Successfully built smart-open
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found ex

In [2]:
pip install pymorphy2[fast]

     |████████████████████████████████| 371 kB 6.9 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=856055 sha256=ab62be153fbc086b9df214da302b0ad4d6b4c480a63284eaff0a13dc785967a1
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [3]:
pip install Gensim

In [4]:
import spacy
import gensim
from gensim.models import Word2Vec 
from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
nlp = spacy.load('ru_core_news_md')

In [6]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt

--2021-05-12 23:25:17--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11655409 (11M) [text/plain]
Saving to: ‘neuroleptic_antidepr2.txt’

neuroleptic_antidep 100%[===================>]  11.12M  27.3MB/s    in 0.4s    

2021-05-12 23:25:18 (27.3 MB/s) - ‘neuroleptic_antidepr2.txt’ saved [11655409/11655409]



In [7]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt

--2021-05-12 23:25:18--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19072917 (18M) [text/plain]
Saving to: ‘rakpobedim_ru2.txt’

rakpobedim_ru2.txt  100%[===================>]  18.19M  48.9MB/s    in 0.4s    

2021-05-12 23:25:19 (48.9 MB/s) - ‘rakpobedim_ru2.txt’ saved [19072917/19072917]



In [8]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt

--2021-05-12 23:25:19--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18273113 (17M) [text/plain]
Saving to: ‘hv-info_gepatit2.txt’

hv-info_gepatit2.tx 100%[===================>]  17.43M  47.9MB/s    in 0.4s    

2021-05-12 23:25:20 (47.9 MB/s) - ‘hv-info_gepatit2.txt’ saved [18273113/18273113]



In [9]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz

--2021-05-12 23:25:20--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140310878 (134M) [application/x-gzip]
Saving to: ‘news_upos_cbow_300_2_2017.bin.gz’

news_upos_cbow_300_ 100%[===================>] 133.81M  25.3MB/s    in 6.1s    

2021-05-12 23:25:27 (21.9 MB/s) - ‘news_upos_cbow_300_2_2017.bin.gz’ saved [140310878/140310878]



In [10]:
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()


In [11]:
handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [12]:
handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [13]:
len(my_txt)

27946773

In [14]:
tfidf = TfidfVectorizer(max_features=3000, ngram_range=(1,3))

In [15]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz

--2021-05-12 23:25:27--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140310878 (134M) [application/x-gzip]
Saving to: ‘news_upos_cbow_300_2_2017.bin.gz.1’

news_upos_cbow_300_ 100%[===================>] 133.81M  23.9MB/s    in 6.5s    

2021-05-12 23:25:34 (20.5 MB/s) - ‘news_upos_cbow_300_2_2017.bin.gz.1’ saved [140310878/140310878]



In [16]:
posts = my_txt.split('----')

In [17]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]

vectorized_txt = tfidf.fit_transform(sentences_tokenized)
sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]

In [18]:
from gensim.models import FastText
from google.colab import files

In [19]:
ft = FastText(sentences_tokenized, size=100, window=5, min_count=5, workers=4,sg=1)

In [20]:
ft.save('ft_model')

In [21]:
files.download('ft_model')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
ft.most_similar('алимемазин')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('левомепромазин', 0.8782601356506348),
 ('триоксазин', 0.8759325742721558),
 ('трифлуоперазин', 0.8414560556411743),
 ('хлорпромазин', 0.840761661529541),
 ('пипофезин', 0.8369158506393433),
 ('зимелидин', 0.8339601755142212),
 ('тиоридазин', 0.8336536884307861),
 ('ипрониазид', 0.8322772979736328),
 ('палиперидон', 0.8278595209121704),
 ('изокарбоксазид', 0.8273183703422546)]

In [23]:
ft.wv.__getitem__('алимемазин')

array([-0.2453686 ,  0.56710815,  0.06693438, -0.16957112,  0.18656401,
        0.16413006,  0.32660273, -0.3223524 , -0.03784393, -0.24079105,
       -0.2768778 , -0.03266587, -0.28471243, -0.13557324, -0.49659124,
        0.2568454 ,  0.10839053, -0.00324988, -0.03547192, -0.07265068,
        0.23425522, -0.04839876, -0.4392015 ,  0.055676  ,  0.1707488 ,
        0.01938497, -0.14036372,  0.13370116, -0.14899142, -0.51764774,
       -0.11587782,  0.17604667,  0.44437551,  0.5519726 ,  0.25264975,
        0.07480077,  0.7937876 , -0.25629357, -0.17646489,  0.5023675 ,
       -0.45949692, -0.15450826, -0.47792   ,  0.22126837, -0.27460843,
       -0.3074345 , -0.0726065 , -0.12907557, -0.3015485 , -0.08681021,
       -0.5983358 ,  0.7649138 , -0.20182474, -0.52963454,  0.2898467 ,
        0.17150718, -0.41079336, -0.41922945,  0.2998531 ,  0.29475224,
       -0.42352965,  0.21196015,  0.3379076 , -0.20853049, -0.22989003,
       -0.16098188,  0.4679046 ,  0.15750182,  0.22258195, -0.01

In [24]:
# files.download('small_fasttext60k')

In [25]:
# w2v = gensim.models.KeyedVectors.load_word2vec_format('news_upos_cbow_300_2_2017.bin.gz', binary=True)

In [26]:
# model = Word2Vec(size=300, min_count=1)
# model.build_vocab(sentences_tokenized)
# total_examples = model.corpus_count
# model.build_vocab([list(w2v.vocab.keys())], update=True)
# model.intersect_word2vec_format("news_upos_cbow_300_2_2017.bin.gz", binary=True)
# model.train(sentences_tokenized, total_examples=total_examples, epochs=model.iter)

In [27]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('алимемазин')), \
                  np.atleast_2d(ft.wv.get_vector('азалептин')))[0][0]

0.70144963

In [28]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('алимемазин')), \
                  np.atleast_2d(ft.wv.get_vector('стул')))[0][0]

0.32482818

In [29]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('дак')), \
                  np.atleast_2d(ft.wv.get_vector('соф')))[0][0]

0.9281227

In [30]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('дак')), \
                  np.atleast_2d(ft.wv.get_vector('софосбувир')))[0][0]

0.5488733

In [31]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('соф')), \
                  np.atleast_2d(ft.wv.get_vector('софосбувир')))[0][0]

0.53131956

In [32]:
# Модель есть, теперь надо набрать корпус из трех форумов примерно поровну, и чтобы не помереть при разметке.

СОВСЕМ НЕМНОЖКО, для начала!!


In [33]:
short_txt = " "
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
short_txt = handler.read()[:60000]
handler.close()

handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
short_txt += handler.read()[:60000]
handler.close()

handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
short_txt += handler.read()[:60000]
handler.close()

In [34]:
len(short_txt)

180000

In [35]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [36]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [37]:
sample = 'он'
print('Одуш', morph.parse(sample)[0].tag.animacy)
print('Падеж', morph.parse(sample)[0].tag.case )
print('Пол', morph.parse(sample)[0].tag.gender)
print('Число', morph.parse(sample)[0].tag.number)
print('Лицо', morph.parse(sample)[0].tag.person)
print('Часть речи', morph.parse(sample)[0].tag.POS)
print('Время', morph.parse(sample)[0].tag.tense)
print('Залог', morph.parse(sample)[0].tag.voice)



Одуш None
Падеж nomn
Пол masc
Число sing
Лицо 3per
Часть речи NPRO
Время None
Залог None


In [38]:
aa = morph.parse(sample)[0]
print(aa.tag)

NPRO,masc,3per,Anph sing,nomn


In [39]:
doc = nlp('У меня практически нет никаких проблем, и в частности, с лекарством.')
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_, 
            [child for child in token.children], morph.parse(token.text)[0].tag,
          [child for child in token.head.children])


У case меня PRON [] PREP [У]
меня obl нет VERB [У] NPRO,1per sing,accs [меня, практически, проблем, лекарством, .]
практически advmod нет VERB [] ADVB [меня, практически, проблем, лекарством, .]
нет ROOT нет VERB [меня, практически, проблем, лекарством, .] PRED,pres [меня, практически, проблем, лекарством, .]
никаких det проблем NOUN [] ADJF,Apro plur,gent [никаких]
проблем nsubj нет VERB [никаких] NOUN,inan,femn plur,gent [меня, практически, проблем, лекарством, .]
, punct лекарством NOUN [] PNCT [,, и, в, с]
и cc лекарством NOUN [] CONJ [,, и, в, с]
в discourse лекарством NOUN [частности, ,] PREP [,, и, в, с]
частности fixed в ADP [] NOUN,inan,femn sing,loct [частности, ,]
, punct в ADP [] PNCT [частности, ,]
с case лекарством NOUN [] PREP [,, и, в, с]
лекарством conj нет VERB [,, и, в, с] NOUN,inan,neut sing,ablt [меня, практически, проблем, лекарством, .]
. punct нет VERB [] PNCT [меня, практически, проблем, лекарством, .]


In [40]:
doc = nlp('У меня практически нет никаких проблем, и в частности, с лекарством.')
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children], morph.parse(token.text)[0].tag,
          [child for child in token.head.children],)


У case меня PRON [] PREP [У]
меня obl нет VERB [У] NPRO,1per sing,accs [меня, практически, проблем, лекарством, .]
практически advmod нет VERB [] ADVB [меня, практически, проблем, лекарством, .]
нет ROOT нет VERB [меня, практически, проблем, лекарством, .] PRED,pres [меня, практически, проблем, лекарством, .]
никаких det проблем NOUN [] ADJF,Apro plur,gent [никаких]
проблем nsubj нет VERB [никаких] NOUN,inan,femn plur,gent [меня, практически, проблем, лекарством, .]
, punct лекарством NOUN [] PNCT [,, и, в, с]
и cc лекарством NOUN [] CONJ [,, и, в, с]
в discourse лекарством NOUN [частности, ,] PREP [,, и, в, с]
частности fixed в ADP [] NOUN,inan,femn sing,loct [частности, ,]
, punct в ADP [] PNCT [частности, ,]
с case лекарством NOUN [] PREP [,, и, в, с]
лекарством conj нет VERB [,, и, в, с] NOUN,inan,neut sing,ablt [меня, практически, проблем, лекарством, .]
. punct нет VERB [] PNCT [меня, практически, проблем, лекарством, .]


In [41]:
sample = 'он'
print('Одуш', morph.parse(sample)[0].tag.animacy)
print('Падеж', morph.parse(sample)[0].tag.case )
print('Пол', morph.parse(sample)[0].tag.gender)
print('Число', morph.parse(sample)[0].tag.number)
print('Лицо', morph.parse(sample)[0].tag.person)
print('Часть речи', morph.parse(sample)[0].tag.POS)
print('Время', morph.parse(sample)[0].tag.tense)
print('Залог', morph.parse(sample)[0].tag.voice)


Одуш None
Падеж nomn
Пол masc
Число sing
Лицо 3per
Часть речи NPRO
Время None
Залог None


***Разметка для кореференции***

In [42]:
topic_list = short_txt.split('\nTOPIC')

In [43]:
import re
print(re.search(r'[A-Za-zА-Яа-я]', ','))

None


In [44]:
# sentences_tokenized = [w.lower() for w in short_posts]
# sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]
# sentences_tokenized[2]

In [45]:
print(len(topic_list))
topic_list[1]

23


'\nОбсуждаем препарат Рисперидон (Рисполепт, Сперидан)\n----\nНЕ знаю, стоило ли мне писать. У меня практически нет никаких проблем, и в частности, с лекарством. По крайней мере врач говорит, что всё идёт как надо. У меня с детства бывают состояния, когда я без видимых причин впадаю в панику, не управляю своими действиями. Странное беспокойство может продолжаться от нескольких мгновений до нескольких суток. Как-то удавалось жить и с этим мириться. Каждый раз, придя в себя, вынуждена придумывать для окружающих более или менее правдоподобное оправдание своим поступкам. Чувствую себя сторонним наблюдателем собственной жизни. В определённый момент страхи стали в основном связаны с учёбой (а ещё с общением). На последнем курсе училища всё это вылилось в невозможность в общем потоке сдать экзамены.\nОбратились к врачу. Со стороны родителей это делалось с большой неохотой. От чего меня лечат, не уточнили. После пары "вводных бесед" назначили небольшую дозировку рисполепта (сказали "для начала

In [46]:
post_list = []
sent_list = []
for topic_num, topic in enumerate(topic_list):
  for post_num, post in enumerate(topic.split('----')):
    for sent_num, sent in enumerate(sent_tokenize(post)):
      # sent_list.append((tokenizer.tokenize(sent.lower()), sent_num, post_num, topic_num))
      if sent.lower():
        sent_list.append((sent.lower(), sent_num, post_num, topic_num))
# post_list[35] # quote

In [47]:
len(sent_list)

2243

In [48]:
sent_list[0][1]

0

In [49]:
sent_list[10:15]

[('на последнем курсе училища всё это вылилось в невозможность в общем потоке сдать экзамены.',
  9,
  1,
  1),
 ('обратились к врачу.', 10, 1, 1),
 ('со стороны родителей это делалось с большой неохотой.', 11, 1, 1),
 ('от чего меня лечат, не уточнили.', 12, 1, 1),
 ('после пары "вводных бесед" назначили небольшую дозировку рисполепта (сказали "для начала"...).',
  13,
  1,
  1)]

In [50]:
sent = 'я сказал(а) ты пидор'
is_ans = 0

for i in ['сказал(а)', ') писал:', 'писал(а): ↑']:
  is_ans = is_ans + (1 if i in sent else 0) 
is_ans

1

In [51]:
coref_df = pd.DataFrame(columns=['TOKEN', 'TOKEN_VECT', 'IS_ANSWER', 'TOPIC_NUM', 'POST_NUM', 'SENT_NUM', 'ANIMACY', 'CASE', 'GENDER', 'PERSON', 'POS', 'DEPENDENCY', 'HEAD',
                                  'HEAD_ANIMACY', 'HEAD_CASE', 'HEAD_GENDER',  'HEAD_POS', 'HEAD_VECT', 'SENT_VECT', 'HEAD_CHILDS_VECT', 'NER',
                                 'COREFERENCE_CLUSTER', ])

In [52]:
for sent_info in sent_list:

  topic_num = sent_info[3]
  post_num = sent_info[2]
  sent_num = sent_info[1]

  is_ans = 0
  
  sent = sent_info[0]

  is_ans = 0
  for i in ['сказал(а)', ') писал:', 'писал(а): ↑']:
    is_ans = is_ans + (1 if i in sent else 0) 

  sent_tokens = tokenizer.tokenize(sent)
  spacy_tokens = nlp(sent)

  sent_vector = 0
  for token in sent_tokens:
    try:
      sent_vector += ft.wv.__getitem__(token.lower())
      sent_vector = sent_vector/len(sent_tokens) if len(sent_tokens) > 0 else 0
    except KeyError:
      print(token.lower(), 'O__o')
  
  for token in spacy_tokens:
    # if re.search(r'[A-Za-zА-Яа-я]', token.text):
    if tokenizer.tokenize(token.text):
        morphy = morph.parse(token.text)[0]

        app_dict = {}
        app_dict['TOKEN'] = token.text
        try:
          app_dict['TOKEN_VECT'] = ft.wv.__getitem__(token.text.lower()) if token.text else None
        except KeyError:
          print(token.text.lower(), 'O__o')
        app_dict['IS_ANSWER'] = is_ans
        app_dict['TOPIC_NUM'] = topic_num 
        app_dict['POST_NUM'] = post_num 
        app_dict['SENT_NUM'] = sent_num 

        app_dict['ANIMACY'] = morphy.tag.animacy
        app_dict['CASE'] = morphy.tag.case
        app_dict['GENDER'] = morphy.tag.gender
        app_dict['PERSON'] = morphy.tag.person
        app_dict['POS'] = morphy.tag.POS
        app_dict['DEPENDENCY'] = token.dep_ 

        app_dict['HEAD'] = token.head.text
        try:
          app_dict['HEAD_VECT'] = ft.wv.__getitem__(token.head.text.lower()) if token.head.text else '-'
        except KeyError:
          app_dict['HEAD_VECT'] = None
          print(token.head.text.lower(), 'here')
        app_dict['HEAD_POS'] = token.head.pos_
        morphy = morph.parse(token.head.text)[0]
        app_dict['HEAD_ANIMACY'] = morphy.tag.animacy
        app_dict['HEAD_CASE'] = morphy.tag.case 
        app_dict['HEAD_GENDER'] = morphy.tag.gender 
        # app_dict['HEAD_PERSON'] = morphy.tag.person or '-'
        app_dict['HEAD_POS'] = morphy.tag.POS
        app_dict['SENT_VECT'] = sent_vector
        try:
          if token.head.children:
            app_dict['HEAD_CHILDS_VECT'] = 0
            for child in token.head.children:
              if re.search(r'[A-Za-zА-Яа-я]', child.text):
                app_dict['HEAD_CHILDS_VECT'] += ft[child.text.lower()]
            if list(token.head.children):
              app_dict['HEAD_CHILDS_VECT'] = app_dict['HEAD_CHILDS_VECT']/len(list(token.head.children))
        except (KeyError, ZeroDivisionError):
          print(child.text.lower(), 'end')

        app_dict['NER'] = 0
        app_dict['COREFERENCE_CLUSTER'] = 0
        coref_df = coref_df.append(app_dict, ignore_index=True)  
    


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


т.к end
т.к end
т.к O__o
т.к end

 here

 here

 here
2-е end
2-е O__o
2-е end
2-е end


 here


 here
т.к end
т.к end
т.к O__o
т.к end

 here

 here

 here
2-е end
2-е O__o
2-е end
2-е end


 here


 here
0,5 O__o
$ here
u.s end
u.s end
u.s end
u.s end
1.4 O__o
$ here
u.s end
u.s O__o
u.s end
u.s end
- here
- here


 here


 here


 here
1.2 O__o


 here


 here

 here
0,5 O__o
0,5 O__o
$ here
u.s end
u.s end
u.s end
u.s end
1.4 O__o
$ here
u.s end
u.s O__o
u.s end
u.s end
- here
- here


 here


 here


 here
1.2 O__o


 here


 here

 here
0,5 O__o
% here
% here
% here
% here
% here
% here
4x7 O__o
4x7 O__o
4x7 O__o


 here


 here

 here


 here
5,5 O__o


 here

 here

 here
2,5 O__o

 here

 here
% here
% here
% here

 here

 here


 here


 here


 here
* here


 here
8) O__o


 here

 here

 here

 here

 here
т.е end
1,5 O__o
т.е end
т.е end
т.е end
т.е end
т.е O__o
т.е end
т.е end
1,5 O__o
т.е end
т.е end
т.е end
т.е end
т.е O__o
т.е end
- here
т.е end
т.е end
- here
- here
т

In [53]:
coref_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,"[-0.35198107, 0.5814818, -0.6922001, -0.654415...",0,1,0,0,None,None,None,1per,VERB,ROOT,обсуждаем,None,None,None,VERB,"[-0.35198107, 0.5814818, -0.6922001, -0.654415...","[-0.005098215, 0.17649734, 0.093845055, -0.053...","[-0.99740636, 0.36573833, -0.7532651, -0.01085...",0,0
1,препарат,"[-0.99740636, 0.36573833, -0.7532651, -0.01085...",0,1,0,0,inan,nomn,masc,None,NOUN,obj,обсуждаем,None,None,None,VERB,"[-0.35198107, 0.5814818, -0.6922001, -0.654415...","[-0.005098215, 0.17649734, 0.093845055, -0.053...","[-0.99740636, 0.36573833, -0.7532651, -0.01085...",0,0
2,рисперидон,"[-0.21882823, 0.8052027, 0.5637987, -0.1392462...",0,1,0,0,anim,nomn,masc,None,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.99740636, 0.36573833, -0.7532651, -0.01085...","[-0.005098215, 0.17649734, 0.093845055, -0.053...","[-0.21882823, 0.8052027, 0.5637987, -0.1392462...",0,0
3,рисполепт,"[-0.33769298, 0.671512, 0.539861, -0.22888352,...",0,1,0,0,inan,gent,femn,None,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,"[-0.21882823, 0.8052027, 0.5637987, -0.1392462...","[-0.005098215, 0.17649734, 0.093845055, -0.053...","[-0.33769298, 0.671512, 0.539861, -0.22888352,...",0,0
4,сперидан,"[0.05934307, 0.71211994, 0.34583476, -0.213859...",0,1,0,0,None,None,masc,None,PRTS,conj,рисполепт,inan,gent,femn,NOUN,"[-0.33769298, 0.671512, 0.539861, -0.22888352,...","[-0.005098215, 0.17649734, 0.093845055, -0.053...","[0.019781023, 0.2373733, 0.11527825, -0.071286...",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26492,и,"[-0.11929728, -0.0030615432, -0.224329, -0.084...",0,22,68,0,None,None,None,None,CONJ,cc,пошлины,inan,nomn,femn,NOUN,"[-0.18635513, -0.90914214, 0.529626, -1.112177...","[0.02620858, -0.0037238898, -0.031466015, -0.0...","[0.14148605, -0.22999011, -0.32839295, -0.4938...",0,0
26493,таможенные,"[0.5437554, -0.6869088, -0.76084983, -1.396614...",0,22,68,0,None,nomn,None,None,ADJF,amod,пошлины,inan,nomn,femn,NOUN,"[-0.18635513, -0.90914214, 0.529626, -1.112177...","[0.02620858, -0.0037238898, -0.031466015, -0.0...","[0.14148605, -0.22999011, -0.32839295, -0.4938...",0,0
26494,пошлины,"[-0.18635513, -0.90914214, 0.529626, -1.112177...",0,22,68,0,inan,nomn,femn,None,NOUN,nsubj,есть,None,None,None,INFN,"[-1.0104536, -0.5207962, -0.12965627, -1.68644...","[0.02620858, -0.0037238898, -0.031466015, -0.0...","[-0.27050188, -0.49391568, 0.05964191, -0.7663...",0,0
26495,как,"[0.30085436, 0.047424514, -0.38781196, -0.9260...",0,22,68,0,None,None,None,None,CONJ,advmod,есть,None,None,None,INFN,"[-1.0104536, -0.5207962, -0.12965627, -1.68644...","[0.02620858, -0.0037238898, -0.031466015, -0.0...","[-0.27050188, -0.49391568, 0.05964191, -0.7663...",0,0


In [54]:
coref_df_anns = coref_df.loc[:, ['TOKEN', 'TOPIC_NUM', 'POST_NUM', 'SENT_NUM', 'IS_ANSWER', 'HEAD', 'NER', 'COREFERENCE_CLUSTER' ]]

In [55]:
coref_df.to_csv('coref_fasttext60k.csv', sep=' ', index=False)

In [56]:
coref_df_anns.to_csv('coref_df_anns60k.csv', sep=' ', index=False)

In [57]:
from google.colab import files

In [58]:
files.download('coref_fasttext60k.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
files.download('coref_df_anns60k.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [60]:
# for topic_num, topic in topic_list:
#   post_list = topic.split('----')
#   sent_list = [(sent, topic_num) for sent, topic_num in enumerate]
#   for sent in sent_list:
#     sent_vector = 0
#     post_num = sent_dict.get(sent)
#     # sent_vector = sum([model.wv.__getitem__(tok.lower())[-1] for tok in sent.split()])
#     if 'сказал(а)', 'писал(а)' in sent:
#       sent.replace('сказал(а)', 'SOQ')
#     for i in sent.split():
#       if re.search(r'[A-Za-zА-Яа-я]', i):
#         try:     
#           sent_vector += ft.wv.__getitem__(i.lower())
#         except KeyError:
#           print(i.lower(), 111)
#     if len(sent.split()) > 0:
#       sent_vector = sent_vector/len(sent.split())


#     my_sent = nlp(sent)
#     for token in my_sent:
#       if re.search(r'[A-Za-zА-Яа-я]', token.text):
#         morphy = morph.parse(token.text)[0]

#         app_dict = {}
#         app_dict['TOKEN'] = token.text or '-'
#         try:
#           app_dict['TOKEN_VECT'] = ft.wv.__getitem__(token.text.lower()) if token.text else '-'
#         except KeyError:
#           print(token.text.lower(), 'O__o')
#         app_dict['POST_NUM'] = post_num or '-'
#         app_dict['ANIMACY'] = morphy.tag.animacy or '-'
#         app_dict['CASE'] = morphy.tag.case or '-'
#         app_dict['GENDER'] = morphy.tag.gender or '-'
#         app_dict['PERSON'] = morphy.tag.person or '-'
#         app_dict['POS'] = morphy.tag.POS or '-' 
#         app_dict['DEPENDENCY'] = token.dep_ or '-'

#         app_dict['HEAD'] = token.head.text or '-'
#         try:
#           app_dict['HEAD_VECT'] = ft.wv.__getitem__(token.head.text.lower()) if token.head.text else '-'
#         except KeyError:
#           app_dict['HEAD_VECT'] = '-'
#           print(token.head.text.lower(), 'here')
#         app_dict['HEAD_POS'] = token.head.pos_ or '-'
#         morphy = morph.parse(token.head.text)[0] or '-'
#         app_dict['HEAD_ANIMACY'] = morphy.tag.animacy or '-'
#         app_dict['HEAD_CASE'] = morphy.tag.case or '-'
#         app_dict['HEAD_GENDER'] = morphy.tag.gender or '-'
#         app_dict['HEAD_POS'] = morphy.tag.POS or '-'
#         app_dict['SENT_VECTOR'] = sent_vector
#         try:
#           if token.head.children:
#             app_dict['HEAD_CHILDS_VECT'] = 0
#             for child in token.head.children:
#               if re.search(r'[A-Za-zА-Яа-я]', child.text):
#                 app_dict['HEAD_CHILDS_VECT'] += ft[child.text.lower()]
#             if list(token.head.children):
#               app_dict['HEAD_CHILDS_VECT'] = app_dict['HEAD_CHILDS_VECT']/len(list(token.head.children))
#         except (KeyError, ZeroDivisionError):
#           print(child.text.lower(), 'end')

#         app_dict['NER'] = 0
#         app_dict['COREFERENCE_CLUSTER'] = 0
#         coref_df = coref_df.append(app_dict, ignore_index=True)


In [61]:
# for topic in topic_list:
#   sent_list = topic.split('----')
#   for sent in sent_list:
#     sent_vector = 0
#     post_num = sent_dict.get(sent)
#     # sent_vector = sum([model.wv.__getitem__(tok.lower())[-1] for tok in sent.split()])
    
#     for i in sent.split():
#       if re.search(r'[A-Za-zА-Яа-я]', i):
#         try:     
#           sent_vector += ft.wv.__getitem__(i.lower())
#         except KeyError:
#           print(i.lower(), 111)
#     if len(sent.split()) > 0:
#       sent_vector = sent_vector/len(sent.split())

#     # if sent_vector == '-' or None or 0:
#     #   print(sent)
#     my_sent = nlp(sent)
#     for token in my_sent:
#       if re.search(r'[A-Za-zА-Яа-я]', token.text):
#         morphy = morph.parse(token.text)[0]

#         app_dict = {}
#         app_dict['TOKEN'] = token.text or '-'
#         try:
#           app_dict['TOKEN_VECT'] = ft.wv.__getitem__(token.text.lower()) if token.text else '-'
#         except KeyError:
#           print(token.text.lower(), 'O__o')
#         app_dict['POST_NUM'] = post_num or '-'
#         app_dict['ANIMACY'] = morphy.tag.animacy or '-'
#         app_dict['CASE'] = morphy.tag.case or '-'
#         app_dict['GENDER'] = morphy.tag.gender or '-'
#         app_dict['PERSON'] = morphy.tag.person or '-'
#         app_dict['POS'] = morphy.tag.POS or '-' 
#         app_dict['DEPENDENCY'] = token.dep_ or '-'

#         app_dict['HEAD'] = token.head.text or '-'
#         try:
#           app_dict['HEAD_VECT'] = ft.wv.__getitem__(token.head.text.lower()) if token.head.text else '-'
#         except KeyError:
#           app_dict['HEAD_VECT'] = '-'
#           print(token.head.text.lower(), 'here')
#         app_dict['HEAD_POS'] = token.head.pos_ or '-'
#         morphy = morph.parse(token.head.text)[0] or '-'
#         app_dict['HEAD_ANIMACY'] = morphy.tag.animacy or '-'
#         app_dict['HEAD_CASE'] = morphy.tag.case or '-'
#         app_dict['HEAD_GENDER'] = morphy.tag.gender or '-'
#         # app_dict['HEAD_PERSON'] = morphy.tag.person or '-'
#         app_dict['HEAD_POS'] = morphy.tag.POS or '-'
#         app_dict['SENT_VECTOR'] = sent_vector
#         try:
#           if token.head.children:
#             app_dict['HEAD_CHILDS_VECT'] = 0
#             for child in token.head.children:
#               if re.search(r'[A-Za-zА-Яа-я]', child.text):
#                 app_dict['HEAD_CHILDS_VECT'] += ft[child.text.lower()]
#             if list(token.head.children):
#               app_dict['HEAD_CHILDS_VECT'] = app_dict['HEAD_CHILDS_VECT']/len(list(token.head.children))
#         except (KeyError, ZeroDivisionError):
#           print(child.text.lower(), 'end')

#         app_dict['NER'] = 0
#         app_dict['COREFERENCE_CLUSTER'] = 0
#         coref_df = coref_df.append(app_dict, ignore_index=True)
